In [1]:
pip uninstall -y tensorflow

Found existing installation: tensorflow 2.16.1
Uninstalling tensorflow-2.16.1:
  Successfully uninstalled tensorflow-2.16.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.0/230.0 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.6 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

In [3]:
import numpy as np
import pandas as pd
import random
import torch
from torch.optim import Adam
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
from transformers import BertTokenizer,BertConfig
from transformers import BertForMaskedLM

In [4]:
df = pd.read_csv("/kaggle/input/new-suicide/Suicide_Detection.csv")

# Pretrain Bert_API 

* Prepare dataset

In [13]:
# balance dataset pretrain
dataset = pd.concat([df[df['class'] == 'suicide'][20000:30000], df[df['class'] == 'non-suicide'][20000:30000]], axis = 0)

In [14]:
# mlm task dataset
def prepare_data_for_mlm(df, tokenizer, max_length= 300):
  texts = df['text'].tolist()
  inputs = tokenizer(texts, max_length=max_length, padding='max_length', truncation=True)
  return inputs

tokenizer = BertTokenizer.from_pretrained('/kaggle/input/pretrained-v3')
train_inputs = prepare_data_for_mlm(dataset, tokenizer)

# Create a PyTorch Dataset for MLM
class MLMDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = MLMDataset(train_inputs)

# Create a DataCollator for MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

# Create a DataLoader for MLM
train_dataloader = DataLoader(
    train_dataset, batch_size= 64, collate_fn=data_collator
)

* load model pretrain

In [15]:
# load BERT for MLM
config = BertConfig.from_pretrained("/kaggle/input/pretrained-v3", num_hidden_layers=1)
model = BertForMaskedLM.from_pretrained("/kaggle/input/pretrained-v3", config=config)

In [16]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

* training

In [17]:
# convert to GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# optimizer
epochs = 11
optimizer = Adam(model.parameters(), lr=0.002)
store_loss = {'loss MLM': []}
# training
model.train()
for epoch in range(epochs):
    train_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}"):
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    if epoch == 10:
        model.save_pretrained("suicide-bert-pretrained")
        tokenizer.save_pretrained("suicide-bert-pretrained")
    store_loss['loss MLM'].append(train_loss / len(train_dataloader))
    print(f"Epoch {epoch + 1}: Pretrain Loss: {train_loss / len(train_dataloader)}")

Training Epoch 1: 100%|██████████| 313/313 [05:55<00:00,  1.14s/it]


Epoch 1: Pretrain Loss: 3.517213663734948


Training Epoch 2: 100%|██████████| 313/313 [05:55<00:00,  1.13s/it]


Epoch 2: Pretrain Loss: 3.400194068305409


Training Epoch 3: 100%|██████████| 313/313 [05:54<00:00,  1.13s/it]


...

Epoch 9: Pretrain Loss: 3.01453208009275


Training Epoch 10: 100%|██████████| 313/313 [05:55<00:00,  1.14s/it]


Epoch 10: Pretrain Loss: 2.9875987177839676


Training Epoch 11: 100%|██████████| 313/313 [05:55<00:00,  1.14s/it]


Epoch 11: Pretrain Loss: 2.9521815761590537


* save model pretrain

In [18]:
model.save_pretrained("suicide-bert-pretrained-1")
tokenizer.save_pretrained("suicide-bert-pretrained-1")

('suicide-bert-pretrained-1/tokenizer_config.json',
 'suicide-bert-pretrained-1/special_tokens_map.json',
 'suicide-bert-pretrained-1/vocab.txt',
 'suicide-bert-pretrained-1/added_tokens.json')

# Finetune 

* prepare data

In [6]:
# balance dataset pretrain
dataset = pd.concat([df[df['class'] == 'suicide'][:5000], df[df['class'] == 'non-suicide'][:5000]], axis = 0)

In [5]:
texts = df['text'].tolist()
labels = df['class'].tolist()

In [11]:
from sklearn.model_selection import train_test_split

# load tokenizer
tokenizer = BertTokenizer.from_pretrained("/kaggle/input/pretrained-v3")

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.4)

# encoding text
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [12]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = [1 if label == 'suicide' else 0 for label in labels]

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

finetune_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

finetune_loader = DataLoader(finetune_dataset, batch_size= 64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size= 64, shuffle=True)

* load model pretrained for finetune 

In [13]:
from transformers import BertForSequenceClassification

# load pretrained
config = BertConfig.from_pretrained("/kaggle/input/pretrained-v3", num_hidden_layers=1,num_labels=2)
model = BertForSequenceClassification.from_pretrained("/kaggle/input/pretrained-v3",config = config)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/pretrained-v3 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


* training

In [ ]:
epochs = 10
optimizer = Adam(model.parameters(), lr=2e-3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(finetune_loader, desc=f"Training Epoch {epoch + 1}"):
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        train_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_train_loss = train_loss / len(finetune_loader)
    print(f"Epoch {epoch + 1} - Average Training Loss: {avg_train_loss:.4f}")

    # evaluate 
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == batch['labels']).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    accuracy = correct / len(val_dataset)
    print(f"Epoch {epoch + 1} - Validation Loss: {avg_val_loss:.4f} - Accuracy: {accuracy:.4f}")

Training Epoch 1: 100%|██████████| 2176/2176 [17:06<00:00,  2.12it/s]


Epoch 1 - Average Training Loss: 0.1482
Epoch 1 - Validation Loss: 0.1171 - Accuracy: 0.9565


Training Epoch 2: 100%|██████████| 2176/2176 [17:04<00:00,  2.12it/s]


Epoch 2 - Average Training Loss: 0.1007
Epoch 2 - Validation Loss: 0.1169 - Accuracy: 0.9570


Training Epoch 3: 100%|██████████| 2176/2176 [17:08<00:00,  2.12it/s]


...

Epoch 8 - Average Training Loss: 0.0293
Epoch 8 - Validation Loss: 0.2151 - Accuracy: 0.9519


Training Epoch 9: 100%|██████████| 2176/2176 [17:16<00:00,  2.10it/s]


Epoch 9 - Average Training Loss: 0.0272
Epoch 9 - Validation Loss: 0.2011 - Accuracy: 0.9516


Training Epoch 10: 100%|██████████| 2176/2176 [17:18<00:00,  2.09it/s]


Epoch 10 - Average Training Loss: 0.0210
Epoch 10 - Validation Loss: 0.1936 - Accuracy: 0.9496


* save model

In [15]:
model.save_pretrained("bert-classify-model")
tokenizer.save_pretrained("bert-classify-model")

('bert-classify-model/tokenizer_config.json',
 'bert-classify-model/special_tokens_map.json',
 'bert-classify-model/vocab.txt',
 'bert-classify-model/added_tokens.json')